# Install and Load Needed Modules

In [2]:
!pip install transformers

In [3]:
!pip install langchain 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.9/815.9 kB 15.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 52.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.0 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cubinlinker, which is not installed.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 23.8.0 requires ptxcompiler, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 

In [4]:
!pip install sentencepiece

# Load model  and tokenizer

In [5]:
# Load model 
from transformers import AutoTokenizer, AutoModelForCausalLM 
from langchain import HuggingFacePipeline

tokenizer = AutoTokenizer.from_pretrained("Universal-NER/UniNER-7B-all")
model = AutoModelForCausalLM.from_pretrained("Universal-NER/UniNER-7B-all")

tokenizer_config.json:   0%|          | 0.00/766 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

...

pytorch_model-00002-of-00003.bin:   0%|          | 0.00/9.89G [00:00<?, ?B/s]

pytorch_model-00003-of-00003.bin:   0%|          | 0.00/7.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [6]:
import torch
from transformers import pipeline
from langchain import HuggingFacePipeline
ner_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer,max_length=1000,trust_remote_code=True,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,)
llm = HuggingFacePipeline(pipeline = ner_pipeline, model_kwargs = {'temperature':0})

2024-02-18 15:04:57.072370: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-18 15:04:57.072494: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-18 15:04:57.203816: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Prompt Template

In [7]:
from langchain import PromptTemplate,  LLMChain


prompt_template = """A virtual assistant answers questions from a user based on the provided text.
USER: Text: {input_text}
ASSISTANT: I’ve read this text.
USER: What describes {entity_name} in the text?
ASSISTANT:
"""
prompt = PromptTemplate(
    input_variables=["input_text", "entity_name"],
    template=prompt_template
)

In [8]:
llm_chain = LLMChain(prompt=prompt, llm=llm)

# Load Text from Json File

In [9]:
import json
# Load your JSON data
with open('/kaggle/input/scraped-pdf/scraped_data.json', 'r', encoding='utf-8') as f:
    data = json.load(f)
text = data['text']  # Assuming the key for your text in JSON is 'text'

# Split PDF Text into Chunks to Respect the model Input Max Length

In [10]:
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
     chunk_size = 500,
     chunk_overlap  = 32,
     length_function = len,
 )
texts = text_splitter.split_text(text)

In [11]:
texts[0]

'Impossible Distillation: from Low-Quality Model to High-Quality Dataset & Model for Summarization and Paraphrasing Jaehun Jung† Peter West†‡ Liwei Jiang†‡ Faeze Brahman†‡ Ximing Lu†‡ Jillian Fisher† Taylor Sorensen† Yejin Choi†‡ †Paul G. Allen School of Computer Science & Engineering, University of Washington ‡Allen Institute for Artificial Intelligence hoony123@cs.washington.edu Abstract It is commonly perceived that the strongest language models (LMs) rely on a combination of massive scale,'

# Test

In [12]:
entity_type='Person'

In [13]:
print(llm_chain.run({"input_text":texts[0],"entity_name":entity_type}))

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Jaehun Jung\u00b2 Peter West\u00b2\u00a3 Liwei Jiang\u00b2\u00a3 Faeze Brahman\u00b2\u00a3 Ximing Lu\u00b2\u00a3 Jillian Fisher\u00b2 Taylor Sorensen\u00b2 Yejin Choi\u00b2\u00a3 hoony123@cs.washington.edu"]


In [16]:
text ="""
My name is Robinson. I am from England. I am eighteen years old. My father is German. My mother is English. I have two brothers. I have one sister. We are a good family.

My father is a good businessman. We have money. I have a good school. I have a good life. But I have a dream. I want to travel. I want to see the world.

My father is a good man. He wants the best for me. But my dream is not his dream. He is not happy. The situation is not easy.

It is morning. My father is in his room. He wants to speak with me. I go to his room. My father has some questions. My father wants to know why I want to travel.

My father says, “Travelling is dangerous. Travelling is not comfortable. You can die.”

I think about it. Travelling is dangerous. It is not comfortable. I can die. But it is also very interesting. I can see new countries. I can meet new people.

I think about travelling every day. Maybe my mother can help me.

I speak with my mother. I tell my mother, “I want to travel. I want to see the world. Please, speak with my father. Maybe with your help, I can travel.”

My mother loves me. But she thinks that travelling is dangerous. She thinks that my place is at home. She thinks that life in England is the best for me.

One year later, I visit one big town. The name of this town is Hull.

I meet my friend. His father has a ship. They travel to London. My friend tells me, “Go with me.”

I think about it. I am not prepared. But I can try if travelling is for me.

I go to London. I am very happy. It is my first long journey. It is very interesting. The first hour is good. But then we have problems. We see a storm. The storm is big.

The wind is strong. The ship goes up and down. I have fear. I think about my home. I think about my comfortable bed.

The storm is finished in the morning. The weather is nice. And the evening is beautiful. Everything is quiet.

One man comes to me. We speak. We speak about the storm. I speak about my fear. The man gives me a drink. I drink very much. I sleep very well at night.

The next day, I forget about my home. I forget about my comfortable bed. I dream more about travelling.
"""
entity_type="Location"
print(llm_chain.run({"input_text":text,"entity_name":entity_type}))

[", England", "the world", "England", "Hull", "London"]
